In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from Mydataset import RDataset
from modle import ResNet1, ResNet2, ResNet3, ResNet4, GAPFC, MWBlock1, MWBlock2, MWBlock3
from centerloss import CenterLoss
import os

# Define the model
class mmfCNN(nn.Module):
    def __init__(self):
        super(mmfCNN, self).__init__()
        self.Rfeature1 = ResNet1()
        self.Rfeature2 = ResNet2()
        self.Rfeature3 = ResNet3()
        self.TOTALfeature1 = ResNet4()
        self.classifier = GAPFC(1)

    def extract_features(self, x1):
        x1 = self.Rfeature1(x1)
        x1 = self.Rfeature2(x1)
        x1 = self.Rfeature3(x1)
        features = self.TOTALfeature1(x1)
        return features

    def forward(self, x1):
        features = self.extract_features(x1)
        output = self.classifier(features)
        return output, features

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
RDI = mmfCNN().to(device)
model_path = './s_trained_model.pth'
RDI.load_state_dict(torch.load(model_path))
RDI.eval()

# Data preprocessing and loading
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

R_root = './data/RGB'
dataset_test = RDataset(R_root, transform, step='test')

num_workers = min(4, os.cpu_count() // 2)
dataload_test = DataLoader(dataset_test, batch_size=512, shuffle=True, drop_last=True, pin_memory=True, prefetch_factor=3, num_workers=num_workers)

# TensorBoard
writer = SummaryWriter(log_dir='tf-logs')

# Testing
correct_sum = 0
total_test_step = 0
confidence_values = []
true_negatives = 0
false_positives = 0
true_positives = 0
false_negatives = 0

with torch.no_grad():
    for test_batch_idx, (data1, label) in enumerate(dataload_test):
        data1, label = data1.to(device), label.to(device)
        outputs, features = RDI(data1)
        outputs = torch.squeeze(outputs)
        confidence = torch.sigmoid(outputs)
        mask = confidence.ge(0.5).float()

        confidence_values.extend(confidence.cpu().numpy())
        correct = (mask == label).sum().item()

        true_negatives += ((mask == 1) & (label == 1)).sum().item()
        false_positives += ((mask == 0) & (label == 1)).sum().item()
        true_positives += ((mask == 0) & (label == 0)).sum().item()
        false_negatives += ((mask == 1) & (label == 0)).sum().item()

        correct_sum += correct
        total_test_step += 1

        metadata = [f"Label: {label[i].item()}, confidence: {confidence[i].item():.4f}" for i in range(label.size(0))]
        writer.add_embedding(features, metadata=metadata, global_step=total_test_step)

# Log confidence values
confidence_values_np = np.array(confidence_values)
writer.add_histogram('Confidence Values', confidence_values_np, bins='auto')

# Compute APCER, NPCER, and ACER
n_attacks = false_positives + true_negatives
n_normals = true_positives + false_negatives

apcer = false_positives / n_attacks if n_attacks > 0 else 0
npcer = false_negatives / n_normals if n_normals > 0 else 0
acer = (apcer + npcer) / 2

print(f"APCER: {apcer:.4f}, NPCER: {npcer:.4f}, ACER: {acer:.4f}")

ave_acc = correct_sum / (total_test_step * dataload_test.batch_size)
print(f"Test Ave Acc: {ave_acc:.4f}")

# Close TensorBoard writer
writer.close()

torch.cuda.empty_cache()

APCER: 0.0374, NPCER: 0.0000, ACER: 0.0187
Test Ave Acc: 0.9828
